# **Roadmap**

1. Install the TensorFlow Object Detection API
2. Setup folder structure
3. Generate the TFRecord files required for training
4. Edit the model pipeline config file and download the pre-trained model checkpoint
5. Train and evaluate the model

⚠️ This notebook is meant to be run in Google Colab for training in order to use GPU capacity.

In [ ]:
# Check GPU setup

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 29 16:18:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check RAM setup

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# **Import libraries**

In [2]:
!pip install fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 2.1 MB/s 
     |████████████████████████████████| 47 kB 5.1 MB/s 
     |████████████████████████████████| 106 kB 55.3 MB/s 
     |████████████████████████████████| 79.9 MB 60.9 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 564 kB 44.5 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 132 kB 60.9 MB/s 
     |████████████████████████████████| 133 kB 66.6 MB/s 
     |████████████████████████████████| 29.2 MB 1.3 MB/s 
     |████████████████████████████████| 135 kB 50.9 MB/s 
     |████████████████████████████████| 61 kB 202 kB/s 
     |████████████████████████████████| 226 kB 51.8 MB/s 
     |████████████████████████████████| 508 kB 37.3 MB/s 
     |████████████████████████████████| 80 kB 10.3 MB/s 
     |████████████████████████████████| 

In [3]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import json
import tensorflow as tf
import json
import shutil
import fiftyone
print(tf.__version__)

Migrating database to v0.17.2


INFO:fiftyone.migrations.runner:Migrating database to v0.17.2


2.8.2


# 1. Create customTF2, training and data folders in your google drive

Create a folder named ***customTF2***.

Create another folder named ***training*** inside the ***customTF2*** folder
(***training*** folder is where the checkpoints will be saved during training)

Create another folder named ***data*** inside the ***customTF2*** folder.

# 2. Mount drive and link your folder

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/Othercomputers/My_MacBook_Pro_Machine_Learning/ /MacBook
!ls /MacBook

Mounted at /content/gdrive
customTF2  Final_Dataset_Test  Final_Dataset_Train  notebooks


# 3. Clone the tensorflow models git repository & Install TensorFlow Object Detection API

In [11]:
# clone the tensorflow models on the colab cloud vm
!git clone --q https://github.com/tensorflow/models.git

#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/models/research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.9 MB 2.5 MB/s 
     |████████████████████████████████| 352 kB 66.1 MB/s 
     |████████████████████████████████| 2.2 MB 37.6 MB/s 
     |████████████████████████████████| 25.0 MB 637 kB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 116 kB 52.8 MB/s 
     |████████████████████████████████| 636 kB 45.8 MB/s 
     |████████████████████████████████| 

In [12]:
# Testing the model builder
!python object_detection/builders/model_builder_tf2_test.py

2022-09-30 10:54:37.045956: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-30 10:54:37.915007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-09-30 10:54:37.915976: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-09-30 10:54:37.916007: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you woul

# 4. Train / Test split

We will use the following repo and script (https://github.com/akarazniewicz/cocosplit) to generate a train & test split from the annotations

In [ ]:
# Command to split 80%

⬛️ python cocosplit.py --having-annotations -s 0.8 '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/final_annotations_coco_V6.json' '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/train.json' '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/test.json'

We then need to generate the train & test data folders accordingly

In [ ]:
# Function to generate the train & test folders

def generate_train_test_folders(json_file,path_to_original_data,path_to_new_split_data,name_of_split):
    with open(json_file) as f:
        json_annotations = json.load(f)
    
    for image in json_annotations['images']:
        path_image = image['file_name']
        os.makedirs(os.path.dirname(f"{path_to_new_split_data}/{name_of_split}/{path_image}"), exist_ok=True)
        shutil.copy2(f"{path_to_original_data}/{path_image}", f"{path_to_new_split_data}/{name_of_split}/{path_image}")
    
    return f"Completed transfer of {name_of_split}"


In [ ]:
# Test folder
generate_train_test_folders('/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/test.json', '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset', '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset_Split', 'test')

'Completed transfer of test'

In [ ]:
# Train folder
generate_train_test_folders('/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset/train.json', '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset', '/Users/julienberthomier/code/AmElmo/Main_Projects/Trash Detector (Wall-E)/data/Final_Dataset_Split', 'train')

'Completed transfer of train'

# 5. Generate label_map.pbtxt file

In [ ]:
# Function used to generate the label_map.pbtxt file from the COCO annotations in JSON

def generate_label_map(path_to_json,path_to_label_map):
    
    with open(path_to_json) as f:
        json_annotations = json.load(f)
    
    pbtxt_content = ""
        
    for category in json_annotations['categories']:
        pbtxt_content = (
            pbtxt_content
            + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(category['id'] + 1, category['name'])
        )
    pbtxt_content = pbtxt_content.strip()
        
    with open(path_to_label_map, "w") as f:
        f.write(pbtxt_content)
        print('Successfully created label_map.pbtxt ')   

In [ ]:
generate_label_map('../data/Final_Dataset/final_annotations_coco_V6.json', '../customTF2/label_map.pbtxt')

Successfully created label_map.pbtxt 


# 6. Convert Train & Test to TFRecord

In [9]:
# CLI command using the fiftyone library to turn test set into TFRecord

!fiftyone convert \
            --input-dir /MacBook/Final_Dataset_Test \
            --input-type fiftyone.types.COCODetectionDataset \
            --output-dir /MacBook/customTF2/data \
            --output-kwargs tf_records_path=test.records \
            --output-type fiftyone.types.TFObjectDetectionDataset

Using input format: fiftyone.types.dataset_types.COCODetectionDataset
Using export format: fiftyone.types.dataset_types.TFObjectDetectionDataset
INFO:fiftyone.utils.data.converters:Using export format: fiftyone.types.dataset_types.TFObjectDetectionDataset
Loading dataset from '/MacBook/Final_Dataset_Test'
INFO:fiftyone.utils.data.converters:Loading dataset from '/MacBook/Final_Dataset_Test'
 100% |███████████████| 1892/1892 [18.7s elapsed, 0s remaining, 365.2 samples/s]      
INFO:eta.core.utils: 100% |███████████████| 1892/1892 [18.7s elapsed, 0s remaining, 365.2 samples/s]      
Exporting dataset to '/MacBook/customTF2/data'
INFO:fiftyone.utils.data.converters:Exporting dataset to '/MacBook/customTF2/data'
Directory '/MacBook/customTF2/data' already exists; export will be merged with existing files
Found multiple fields ['detections', 'segmentations'] with compatible type <class 'fiftyone.core.labels.Detections'>; exporting 'detections'
INFO:fiftyone.core.collections:Found multiple f

In [ ]:
# CLI command using the fiftyone library to turn training set into TFRecord

!fiftyone convert \
            --input-dir /MacBook/Final_Dataset_Train \
            --input-type fiftyone.types.COCODetectionDataset \
            --output-dir /MacBook/customTF2/data \
            --output-kwargs tf_records_path=train.records \
            --output-type fiftyone.types.TFObjectDetectionDataset

Using input format: fiftyone.types.dataset_types.COCODetectionDataset
Using export format: fiftyone.types.dataset_types.TFObjectDetectionDataset
INFO:fiftyone.utils.data.converters:Using export format: fiftyone.types.dataset_types.TFObjectDetectionDataset
Loading dataset from '/MacBook/Final_Dataset_Train'
INFO:fiftyone.utils.data.converters:Loading dataset from '/MacBook/Final_Dataset_Train'
 100% |███████████████| 7567/7567 [1.4m elapsed, 0s remaining, 394.3 samples/s]      
INFO:eta.core.utils: 100% |███████████████| 7567/7567 [1.4m elapsed, 0s remaining, 394.3 samples/s]      
Exporting dataset to '/MacBook/customTF2/data'
INFO:fiftyone.utils.data.converters:Exporting dataset to '/MacBook/customTF2/data'
Directory '/MacBook/customTF2/data' already exists; export will be merged with existing files
Found multiple fields ['detections', 'segmentations'] with compatible type <class 'fiftyone.core.labels.Detections'>; exporting 'detections'
INFO:fiftyone.core.collections:Found multiple f

In [50]:
!python create_coco_tf_record_test.py  --logtostderr \
--train_image_dir="/MacBook/Final_Dataset_Train/data" \
--test_image_dir="/MacBook/Final_Dataset_Test/data" \
--train_annotations_file="/MacBook/Final_Dataset_Train/labels.json" \
--test__annotations_file="/MacBook/Final_Dataset_Test/labels.json" \
--output_dir="/MacBook/customTF2/data"

2022-09-30 11:09:28.629093: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-30 11:09:29.458822: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-09-30 11:09:29.458953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-09-30 11:09:29.458977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you woul

In [47]:
%cd /content/models/research/object_detection/dataset_tools

/content/models/research/object_detection/dataset_tools


In [ ]:
# read the JSON to get the path
# iterate over and copy the file to new folder